In [3]:
import os
os.chdir('/home/s3/hyeryung/mucoco')
import joblib

import string
from tqdm import tqdm
import numpy as np
import pandas as pd
import logging

from new_module.utils.load_ckpt import define_model

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

### Map labels to tokens

- Think of a clever way to save locate tokens (need to map 1,0 with input_ids)
- With the test set (200건) -> run locate & edit -> Evaluate.
- Superset 400건 짜리?을ㅁ 가져갈지 고민하기



나와야할 산출물: input_id가 있는

In [5]:
## 라벨링 결과 불러오기
batch_all_cleaned = pd.read_excel('new_module/locate/batch_all_cleaned.xlsx', index_col=0)
batch_jigsaw = batch_all_cleaned.loc[batch_all_cleaned['source']=='jigsaw'].copy()
batch_gpt2 = batch_all_cleaned.loc[batch_all_cleaned['source']=='gpt2'].copy()

print('batch_jigsaw.shape[0]',batch_jigsaw.shape[0], 'batch_gpt2.shape[0]', batch_gpt2.shape[0])

batch_jigsaw.shape[0] 80 batch_gpt2.shape[0] 230


In [6]:
model = AutoModelForSequenceClassification.from_pretrained('/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-energy-training/step_600_best_checkpoint/')
tokenizer = AutoTokenizer.from_pretrained('/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-energy-training/step_600_best_checkpoint/')

In [7]:
punctuations = string.punctuation + '\n '
print(punctuations)
print(list(punctuations))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
 
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '\n', ' ']


In [8]:

def prepare_locate_labels_v1(label_df, my_tokenizer, saved_input_ids=None, mode='jigsaw', log_level=logging.DEBUG):

  my_input_ids, my_locate_ids, my_texts, my_prompts = [], [], [], []
  logger = logging.getLogger('')
  logger.setLevel(log_level)
  punctuations = string.punctuation + '\n '
  punctuations = list(punctuations)
  punctuations.remove('-') ## some toxic words consist of <word>-<word>

  if mode == 'jigsaw':

    text_ids = label_df.loc[(label_df['type']=='text'), 'text_index'].tolist()
    colname_words = [x for x in label_df.columns if type(x) == int]

    for i in tqdm(text_ids):
      logger.debug(f"======================== text id: {i} ========================")
      row_text = label_df.loc[(label_df['text_index']==i) & (label_df['type']=='text')].copy()
      row_label = label_df.loc[(label_df['text_index']==i) & (label_df['type']=='label')].copy()

      text = row_text.loc[:, 'text'].values[0]
      input_ids = my_tokenizer(text)['input_ids'] # jigsaw 는 tokenizer로 tokenize 해서 쓰기

      words = [x for x in row_text.loc[:, colname_words].values[0].tolist() if x is not np.nan]
      labels = row_label.loc[:, colname_words].values[0].tolist()[:len(words)]

      token_level_labels = []
      j = 0
      for id in input_ids:
        t = my_tokenizer.decode(id)
        while j < len(words):
          if t.strip() in punctuations:
            token_level_labels.append(0)
            logger.debug(f"{t}: {0}")
            break
          elif t.strip() in words[j]:
            token_level_labels.append(labels[j])
            logger.debug(f"{t}: {labels[j]}")
            break
          else:
            j+=1

      my_input_ids.append(input_ids)
      my_locate_ids.append(token_level_labels)
      my_texts.append(row_text['text'].values[0])

  elif mode == 'gpt2':

    text_ids = label_df.loc[(label_df['type']=='text'), 'orig_index'].tolist()
    colname_words = [x for x in label_df.columns if type(x) == int]

    for i in tqdm(text_ids):
      i = int(i)
      logger.debug(f"======================== text id: {i} ========================")
      row_text = label_df.loc[(label_df['orig_index']==i) & (label_df['type']=='text')].copy()
      row_label = label_df.loc[(label_df['orig_index']==i) & (label_df['type']=='label')].copy()
      len_words = int(row_text['num_words_gen'].values[0]) ## for truncation

      input_ids = saved_input_ids[i][0]

      _words = row_text.loc[:, colname_words].values[0].tolist()
      words = [x for x in _words if x is not np.nan] ## added this step since there's some nan's in between words
      words = words[:len_words] ## for truncation
      logger.debug(f"words: {words}")
      labels = [x for i, x in enumerate(row_label.loc[:, colname_words].values[0].tolist()) if _words[i] is not np.nan] ## for truncation
      labels = labels[:len_words]

      token_level_labels = []
      trunc_token_list = []
      j = 0
      for id in input_ids:
        t = my_tokenizer.decode(id)
        if t.strip() != '�':
          while j < len(words):
            if (j == len(words) - 1) and (t != '.') and (t in punctuations): ## . 이 찍히고 나서 나오는 \n, \n 따위를 제거하기 위해서.
              j += 1
            else:
              if t.strip() in punctuations:
                token_level_labels.append(0)
                logger.debug(f"{t}: {0}")
                trunc_token_list.append(id)
                break
              elif t.strip() in words[j]:
                token_level_labels.append(labels[j])
                logger.debug(f"{t}: {labels[j]}")
                trunc_token_list.append(id)
                break
              else:
                j+=1
          # 23/09/04 - below is no longer necessary since I already truncated input ids
          if j == len(words): # escape loop after reaching the end of words
            # break
            token_level_labels.append(0)
            trunc_token_list.append(id)
        else:
          token_level_labels.append(0)
          logger.debug(f"{t}: {0}")
          trunc_token_list.append(id)

      my_input_ids.append(trunc_token_list)
      my_locate_ids.append(token_level_labels)
      my_texts.append(row_text['generation_cleaned'].values[0])
      my_prompts.append(row_text['prompt'].values[0])

  return my_input_ids, my_locate_ids, my_texts, my_prompts

#### gpt2

In [18]:
import torch

In [12]:
testset=pd.read_excel('outputs/toxicity/save-init-gen-all-uniform/testset_10000_gpt_FINAL.xlsx')

In [20]:
testset['tokens']=testset['tokens'].apply(lambda x: torch.LongTensor(eval(x)))

In [24]:
testset['tokens'].values[0][0]

tensor([  435,    12, 43478,    11,   423,   587,  6572,    11,   290,   550,
          587,  3190,  6572,    11,  5176,   284,   262, 19877,   286,   262,
         6318,  4498,  5407,  4991,   357,    37,  4090,   737])

In [21]:
primary_tokenizer = AutoTokenizer.from_pretrained('/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-energy-training/step_2800_best_checkpoint/')

In [35]:
testset['tokens_em']=testset['tokens'].apply(lambda x: tokenizer.encode(primary_tokenizer.decode(x[0]),add_special_tokens=False,return_tensors='pt'))

In [45]:
batch_gpt2.loc[0,'text']

'You’ll need a muffin tin to bake these cookies in, which is a bitch to clean up after, except you can use all the flour and sugar from the quiche instead. I was going to'

In [46]:
testset['generation'].startswith('You'll')

0       al-Sham, have been destroyed, and had been com...
1       al-Sham (see my previous report), include miss...
2       al-Sham, managed to kill 275 Chinese in July a...
3       al-Sham groups, were released: to wait and see...
4       al-Sham (formerly known as al-Nusra Front), ar...
                              ...                        
9995    Subcommittee in 2012, at which he headed a dep...
9996    and Commerce Committee was also convened. The ...
9997    and Commerce Committee member, who has said "h...
9998    and Commerce Committee chairman. He did not re...
9999    and Commerce Committee, a part of the upper ch...
Name: generation, Length: 10000, dtype: object

In [50]:
testset['text']=(testset['prompt']+' ' +testset['generation'])

In [55]:
testset.text.str.startswith('You').isna().sum()

1

In [56]:
testset=testset.loc[~testset.text.str.startswith('You').isna(),:].copy()

In [65]:
testset.loc[testset.text.str.startswith("You’ll need"),'text']

4040    You’ll need a muffin tin to bake these cookies...
4041    You’ll need a muffin tin to bake these cookies...
4042    You’ll need a muffin tin to bake these cookies...
4043    You’ll need a muffin tin to bake these cookies...
4044    You’ll need a muffin tin to bake these cookies...
4045    You’ll need a muffin tin to bake these cookies...
4046    You’ll need a muffin tin to bake these cookies...
4047    You’ll need a muffin tin to bake these cookies...
4048    You’ll need a muffin tin to bake these cookies...
4049    You’ll need a muffin tin to bake these cookies...
Name: text, dtype: object

In [60]:
batch_gpt2.text.values[0]

'You’ll need a muffin tin to bake these cookies in, which is a bitch to clean up after, except you can use all the flour and sugar from the quiche instead. I was going to'

In [69]:
testset.loc[4044,'tokens_em']

tensor([[32594,     7,  2382,    62,    71,     6,  4682,    47,    64,   304,
            70,     5, 15039,     8,  4696,    31,     5,  2677, 11529,  1386,
             4]])

In [36]:
gpt2_input_ids, gpt2_labels, gpt2_texts, gpt2_prompts = prepare_locate_labels_v1(batch_gpt2, tokenizer, testset['tokens_em'].tolist(), mode='gpt2',log_level=logging.DEBUG)

100%|██████████| 115/115 [00:00<00:00, 395.91it/s]


In [71]:
label_data = pd.DataFrame({'input_ids': gpt2_input_ids, 'gpt2_labels': gpt2_labels, 'text': gpt2_texts, 'prompt': gpt2_prompts})

In [72]:
label_data.to_excel('outputs/toxicity/save-init-gen-all-uniform/testset_locate_labels_115_em_tokenizer.xlsx')

In [37]:
testset['tokens_em'].values[0]

tensor([[ 1076,    12, 35796,     6,    33,    57,  4957,     6,     8,    56,
            57,  2198,  4957,     6,  2446,     7,     5, 13141,     9,     5,
          3085,  4681,  2938,  2833,    36,   597,  3603,   322]])

In [70]:
testset.loc[4044,'tokens_em']

tensor([[32594,     7,  2382,    62,    71,     6,  4682,    47,    64,   304,
            70,     5, 15039,     8,  4696,    31,     5,  2677, 11529,  1386,
             4]])

In [30]:
gpt2_input_ids

[[tensor(32594),
  tensor(7),
  tensor(2382),
  tensor(62),
  tensor(71),
  tensor(6),
  tensor(4682),
  tensor(47),
  tensor(64),
  tensor(304),
  tensor(70),
  tensor(5),
  tensor(15039),
  tensor(8),
  tensor(4696),
  tensor(31),
  tensor(5),
  tensor(2677),
  tensor(11529),
  tensor(1386),
  tensor(4)],
 [tensor(15328),
  tensor(2323),
  tensor(5),
  tensor(2378),
  tensor(72),
  tensor(50118),
  tensor(50118),
  tensor(37733),
  tensor(13896),
  tensor(12),
  tensor(37252),
  tensor(6),
  tensor(54),
  tensor(34),
  tensor(8069),
  tensor(5),
  tensor(750),
  tensor(9),
  tensor(5),
  tensor(6207),
  tensor(8391),
  tensor(6),
  tensor(21),
  tensor(540)],
 [tensor(2429),
  tensor(49),
  tensor(2598),
  tensor(2286),
  tensor(72),
  tensor(264),
  tensor(33767),
  tensor(7),
  tensor(69),
  tensor(1441),
  tensor(10641),
  tensor(8),
  tensor(957),
  tensor(8),
  tensor(33767),
  tensor(456),
  tensor(7),
  tensor(69),
  tensor(1441),
  tensor(5420),
  tensor(6)],
 [tensor(47),
  

In [ ]:
joblib.dump(gpt2_input_ids, 'testset_FINAL_gpt2_input_ids.pkl')

['testset_FINAL_gpt2_input_ids.pkl']

In [ ]:
joblib.dump(gpt2_labels, 'testset_FINAL_gpt2_labels.pkl')

['testset_FINAL_gpt2_labels.pkl']

In [ ]:
joblib.dump(gpt2_texts, 'testset_FINAL_gpt2_texts.pkl')

['testset_FINAL_gpt2_texts.pkl']

In [ ]:
## save prompts
joblib.dump(gpt2_prompts, 'testset_FINAL_gpt2_prompts.pkl')

['testset_FINAL_gpt2_prompts.pkl']

In [49]:
## convert the results into a dataframe
orig_indices=batch_gpt2.loc[batch_gpt2['type']=='text', 'orig_index'].astype(int).tolist()
locate_results=pd.DataFrame({'prompt': gpt2_prompts, 'generation': gpt2_texts, 'generation_ids': gpt2_input_ids, 'locate_labels': gpt2_labels}, index=orig_indices)

In [50]:
check=testset_2500.merge(locate_results, left_index=True, right_index=True,how='inner')

In [51]:
check.loc[check['prompt_x']!=check['prompt_y']]

,prompt_x,generation_x,generation_ids_x,prompt_y,generation_y,generation_ids_y,locate_labels


In [52]:
check.loc[check['generation_x']!=check['generation_y']]

,prompt_x,generation_x,generation_ids_x,prompt_y,generation_y,generation_ids_y,locate_labels


In [53]:
check['generation_ids_y']=check['generation_ids_y'].apply(lambda x: torch.stack(x).unsqueeze(0)).values

In [54]:
check.loc[check.generation_ids_x.apply(lambda x: len(x[0]))!= check.generation_ids_y.apply(lambda x: len(x[0])),:]

,prompt_x,generation_x,generation_ids_x,prompt_y,generation_y,generation_ids_y,locate_labels


In [56]:
check.loc[check.generation_ids_x.apply(lambda x: len(x[0]))!= check.locate_labels.apply(lambda x: len(x)),:]

,prompt_x,generation_x,generation_ids_x,prompt_y,generation_y,generation_ids_y,locate_labels


In [61]:
testset_2500=joblib.load('testset_large_gpt.pkl')

In [58]:
testset_2500=testset_2500.merge(locate_results[['locate_labels']],left_index=True,right_index=True,how='left')

In [60]:
joblib.dump(testset_2500,'testset_large_gpt.pkl')

['testset_large_gpt.pkl']

Reformat the testset

In [67]:
prompt_list=testset_2500.prompt.unique().tolist()

In [69]:
_, primary_tokenizer = define_model('/content/drive/MyDrive/Colab Notebooks/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin')

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/vocab.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/tokenizer_config.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/tokenizer_config.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/vocab.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https

DEVICE:  cuda


In [94]:
import json
import math

In [99]:
outf=open('./testset_gpt2_2500.jsonl', 'w')
for source_text in prompt_list:
  source_indices = primary_tokenizer.encode(source_text)
  curr_p_gens=testset_2500.loc[testset_2500['prompt']==source_text, :].reset_index(drop=True)

  for sample_idx, row in curr_p_gens.iterrows():

    if sample_idx == 0:
        output = {
            "prompt":{
                "text":source_text,
                "tokens":source_indices},
            "generations":[{
                "text": row['generation'],
                "tokens": row['generation_ids'][0].tolist()
                }]
        }
        if type(row['locate_labels'])==list: # check if not null
          output['generations'][-1]["locate_labels"] = row['locate_labels']
    else:
        output['generations'].append(
            {
                "text": row['generation'],
                "tokens": row['generation_ids'][0].tolist()
            }
        )
        if type(row['locate_labels'])==list: # check if not null
          output['generations'][-1]["locate_labels"] = row['locate_labels']

    if sample_idx==9:
      json.dump(output, outf)
      outf.write("\n")
      outf.flush()
outf.close()

In [102]:
## read jigsaw data & merge labels

testset_jigsaw=pd.read_json('testset_jigsaw.jsonl',lines=True)

In [104]:
## load labeled data
texts = joblib.load('testset_FINAL_jigsaw_texts.pkl')
labels = joblib.load('testset_FINAL_jigsaw_labels.pkl')
input_ids = joblib.load('testset_FINAL_jigsaw_input_ids.pkl')

In [107]:
labeled_jigsaw=pd.DataFrame({'text': texts, 'input_ids': input_ids, 'locate_labels': labels})

In [109]:
testset_jigsaw=testset_jigsaw.merge(labeled_jigsaw,on='text',how='left')

In [112]:
testset_jigsaw.loc[~testset_jigsaw['input_ids'].isna(),:].shape

(40, 4)

In [115]:
testset_jigsaw.head()


,text,toxicity,input_ids,locate_labels
0,You aren't that bright. Citizens were picking ...,0.500000,NaN,NaN
1,References are constantly being made that ind...,0.800000,NaN,NaN
2,"""...these are the people.."" Those REAL people...",0.833333,NaN,NaN
3,This article is especially hilarious in light ...,0.600000,NaN,NaN
4,Absolutely not! We should never learn to live...,0.500000,NaN,NaN


In [118]:
outf=open('./testset_jigsaw_1960.jsonl', 'w')
testset_jigsaw=testset_jigsaw.reset_index(drop=True)
for sample_idx, row in testset_jigsaw.iterrows():

    output = {
        "source":{
            "text": row['text'],
            "toxicity": row['toxicity']
            }
    }
    if type(row['locate_labels'])==list: # check if not null
      output['source']["locate_labels"] = row['locate_labels']
      output['source']["tokens"] = row['input_ids']
    json.dump(output, outf)
    outf.write("\n")
    outf.flush()
outf.close()

In [114]:
testset_jigsaw.to_json('./testset_jigsaw_1960.jsonl', orient="records", lines=True)